In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Population Growth Evolution and prediction ! 🌍 

![](https://images.unsplash.com/photo-1526470608268-f674ce90ebd4?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1934&q=80)

#### This quick notebook about populatin growth is divided in 4 parts :  
- **Data prepartion + Web scrapping** 📥
- **Global population growth analysis in the world's regions** 🔍
- **Specific case China vs India** 🌏
- **Prediction of the world population for the next years** 📉

# 1) Data prepation 

In [ ]:
PercGrow = pd.read_csv("/kaggle/input/population-growth-annual/API_SP.POP.GROW_DS2_en_csv_v2_2164049.csv")
PercGrow.head(3)

**The first dataset contains the country code and name and one column for each year (from 1960 to 2020)**

In [ ]:
IncomeGroup = pd.read_csv("/kaggle/input/population-growth-annual/Metadata_Country_API_SP.POP.GROW_DS2_en_csv_v2_2164049.csv")
IncomeGroup.head(3)

**The second set of data tells us about the income group of each country as well as the region around the world.**  
**Thanks to the country code, we will be able to join these data to the main dataset if we need it.**

In [ ]:
PercGrow.shape

In [ ]:
#Columns SP.POP.GROW and Indicator Code
print(f"{PercGrow[PercGrow.columns[2:4]].value_counts()}\n") #We can removes this columns because they have a unique value

# Year 1960 and 2020 NaN values 
print(PercGrow["1960"].isnull().sum()) # Many NaN values 
print(PercGrow["2020"].isnull().sum()) # Same 

# Then, we drop all these columns from our dataset 
PercGrow = PercGrow.drop(PercGrow.columns[2:4],axis=1)
PercGrow = PercGrow.drop(["1960","2020"],axis=1)
PercGrow.head()

In [ ]:
PercGrow[PercGrow.columns[5:-1]].isnull().sum()

**There are still NaN values, but by removing the countries with missing values from the year 1961, we end up with a cleaner dataset**

In [ ]:
PercGrow = PercGrow.drop(PercGrow[PercGrow["1961"].isnull() == True].index, axis=0)
PercGrow[PercGrow.columns[5:-1]].isnull().sum()
PercGrow = PercGrow.fillna(0)

**From now, the cases seem to be isolated, so one can claim that one NaN means a 0% increase compared to the previous year. That is, an identical population.**

## ADD Population size with web scrapping

**The % growth tells us how much a population has increased in comparison to the previous year.**  
**However, our dataset does not provide any information on the size of a population.**  
**In order to flesh out our data, I went to look up the population of many countries in 1961 on the site 🌐 :** https://www.bluemarblecitizen.com/world-population/1961  

**With a bit of webscrapping, it is quite easy to bring out the data presented in the table in table form.**

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

url = "https://www.bluemarblecitizen.com/world-population/1961"

try:
    page=requests.get(url)
except:
    print("Problem with the request")

soup=BeautifulSoup(page.content,"html.parser")
results = soup.find_all(class_="popTable")

def cleanhtml(raw_html):
    return re.sub(re.compile('<.*?>'), '/', raw_html).split("//")[1:-1]

dataset = []

for table in results:
    job_section = table.find_all("tr")
    for job in job_section:
        scapped = [item.replace("/","") for item in cleanhtml(str(job))[1:3]] 
        if scapped not in dataset:
            dataset.append(scapped)

d = {str(PercGrow.columns[0]): [item[0] for item in dataset[1:]], 'Population_1961': [int(item[1].replace(",","")) for item in dataset[1:]]}
Population_size_1961 = pd.DataFrame(data=d)
Population_size_1961.head()

In [ ]:
#Left Join on Country Name
TestNaN = pd.merge(PercGrow,Population_size_1961,on=str(PercGrow.columns[0]),how='left')
TestNaN[TestNaN.Population_1961.isnull() == True]["Country Name"].unique()

**So if we only use the countries that have data for their population size, we already lose 75 of them.**  
**We can also see that we have a line : "World"!**  
**Quite interesting, so we will remove it from the data set.**

In [ ]:
World = PercGrow[PercGrow["Country Name"] == "World"].copy()
World["Population_1961"] = 3091843507
World

**In order to know the evolution of the population of each selected countries, we multiply the population of the previous year with the growth percentage of the current year to calculate the population growth.**

**❗ Warning ❗: You have to be careful to round the result to avoid having partial humans...** 

**❗ Warning 2 ❗: The results are not 100% accurate but are still usable for our analysis.**

In [ ]:
PopulationSize = pd.merge(PercGrow,Population_size_1961,on=str(PercGrow.columns[0]),how='inner')

for col in PopulationSize.columns[3:-1]:
    PopulationSize[f"Population_{col}"] = round(PopulationSize[f"Population_{int(col)-1}"] + PopulationSize[f"Population_{int(col)-1}"] * (PopulationSize[col]/100))

PopulationSize.head(5)

**Finally, we just add the regions to our dataset and we are ready to start the analysis. ✅** 

In [ ]:
df = pd.merge(IncomeGroup[IncomeGroup.columns[0:3]],PopulationSize,on="Country Code",how='inner')
df.head(5)

# 2) Growth Evolution arround the World 

**Now that our dataset is ready, we can start by observing the evolution of the last 60 years at the level of the regions of the world but also in its totality.**  
**We will compare both the Population in quantity and the evolution in percentage.**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
colunms = World.columns[3:-1].copy()

for col in colunms:
    World[f"Population_{col}"] = round(World[f"Population_{int(col)-1}"] + World[f"Population_{int(col)-1}"] * (World[col]/100))

    World[World.columns[2:61]].values[0]

In [ ]:
#Set a figure 
plt.style.use('seaborn-whitegrid')
fig, ax_G = plt.subplots(figsize=[20,12])
ax_P = ax_G.twinx()
ax_G.set_xlabel("Year",fontsize=15,fontweight=550,labelpad=15)
ax_G.set_ylabel("% Growth",fontsize=15,fontweight=550,labelpad=15)
ax_P.set_ylabel("Population in Billion",fontsize=15,fontweight=550,labelpad=15)
ax_G.set_title("Evolution of the % Growth and the population in the World from 1961 to 2019",fontsize=20,fontweight=600)
ax_P.set_xticks([i*2 for i in range(0,30)])
ax_P.set_xticklabels(labels  = [1961 + i*2 for i in range(0,30)])

ax_G.plot([int(item) for item in World.columns[2:61].tolist()],World[World.columns[2:61]].values[0],linewidth=3,label="% Growth",color = "#CD9FCC")
ax_P.plot([int(item[-4:]) for item in World.columns[61:].tolist()],World[World.columns[61:]].values[0],linewidth=3,label="Population",color = "#0A014F")

ax_G.set_xticks([1961 + i*2 for i in range(0,30)])
ax_G.set_xticklabels([1961 + i*2 for i in range(0,30)])


**Despite a declining percentage of growth, the number of people on Earth is increasing. ↗**  
**Currently we are: 7.674 Billion 👥**  

**Let's see now according to the regions 🌎🌍🌏 !**  
**I start by grouping the countries into regions using the "groupby" function and calculating their average.** 
## By regions

In [ ]:
Evolution_by_region = df.groupby(["Region"]).mean()
Percentage_years_columns = Evolution_by_region.columns[0:59] #% growth columns 
Population_years_columns = Evolution_by_region.columns[59:] #population size columns 
Evolution_by_region

In [ ]:
#Set a figure 
fig = plt.figure(figsize=[20,12])
plt.yticks(fontsize=10,fontweight=550) 
plt.xticks([i*2 for i in range(0,30)],fontsize=8,fontweight=550)
plt.xlabel("Year",fontsize=15,fontweight=550,labelpad=15)
plt.ylabel("% Growth",fontsize=15,fontweight=550,labelpad=15)
plt.title("Evolution of the % Growth from 1961 to 2019 in the different regions across the world",fontsize=20,fontweight=600)

for index in Evolution_by_region.index:
    plt.plot(Evolution_by_region[Percentage_years_columns].loc[index],label = index,linewidth=3)

plt.legend(fontsize=12)


In [ ]:
print("We can see that overall, the % of growth has decreased over the last 60 years !\n")
print(f"Decrease for each region from 1961 to 2019 : \n\n\n{Evolution_by_region['1961'] - Evolution_by_region['2019']}")
Evolution_by_region[["1961","2019"]]

**With the exception of the Sub-Saharan Africa region, which increased slightly, all regions saw their growth decrease between 0.65 and 2.2%.**  

**This is good news, right? 👍**

In [ ]:
#Set a figure 
fig = plt.figure(figsize=[20,12])
plt.yticks(fontsize=10,fontweight=550) 
plt.xticks([i*2 for i in range(0,30)],labels=[1961+i*2 for i in range(0,30)],fontsize=8,fontweight=550)
plt.xlabel("Year",fontsize=15,fontweight=550,labelpad=15)
plt.ylabel("Growth in Billion",fontsize=15,fontweight=550,labelpad=15)
plt.title("Evolution of the population from 1961 to 2019 in the different regions across the world",fontsize=20,fontweight=600)

for index in Evolution_by_region.index:
    plt.plot(df.groupby(['Region']).sum()[Population_years_columns].loc[index],label = index,linewidth=3)

plt.legend(fontsize=12)


In [ ]:
print("On the contrary, despite a decrease of % of growth, the world population has increased for each region ! \n")
print(f"Population Evolution from 1961 to 2019 : \n\n\n{df.groupby(['Region']).sum()['Population_2019'] - df.groupby(['Region']).sum()['Population_1961']}")
df.groupby(["Region"]).sum()[["Population_1961","Population_2019"]]

In [ ]:
result = sum(df.groupby(["Region"]).sum()["Population_2019"].iloc[[0,5]])
print(f"If we add 'East Asia & Pacific' and 'South Asia', we can observe that a lot of people lives in these region :\n{result}")
print("Soit 4 billions peoples, more than half of the total earth population")

Essayons de regarder les "mastodontes" de ces deux regions

In [ ]:
EastAsia = df[df["Region"] == 'East Asia & Pacific'].copy()
SouthAsia = df[df["Region"] == 'South Asia'].copy()
print(SouthAsia[SouthAsia["Population_2019"] == SouthAsia["Population_2019"].max()]["Country Name"])
print(EastAsia[EastAsia["Population_2019"] == EastAsia["Population_2019"].max()]["Country Name"])

# 3) India vs China 

**For each of these regions, one country has more than one billion inhabitants at present.**  
**China and India, often referred to as the massive population places.**

In [ ]:
#Create datasets 
China = df[df["Country Name"] == "China"]
India = df[df["Country Name"] == "India"]

In [ ]:
#Set a figure 
fig, ax_G = plt.subplots(figsize=[20,12])
ax_P = ax_G.twinx()
fig = plt.figure()
ax_G.set_xlabel("Year",fontsize=15,fontweight=550,labelpad=15)
ax_G.set_ylabel("% Growth",fontsize=15,fontweight=550,labelpad=15)
ax_P.set_ylabel("Population in Billion",fontsize=15,fontweight=550,labelpad=15)
ax_G.set_title("Evolution of the % Growth and the population in China and India from 1961 to 2019",fontsize=20,fontweight=600)
ax_P.set_xticks([i*2 for i in range(0,30)])
ax_P.set_xticklabels(labels  = [1961 + i*2 for i in range(0,30)])

ax_G.plot([int(item) for item in China.columns[4:63].tolist()],China[China.columns[4:63]].values[0],linewidth=3,label="% Growth China",color = "#ff758f")
ax_P.plot([int(item[-4:]) for item in China.columns[63:].tolist()],China[China.columns[63:]].values[0],linewidth=3,label="Population China",color = "#a4133c")
ax_G.plot([int(item) for item in India.columns[4:63].tolist()],India[India.columns[4:63]].values[0],linewidth=3,label="% Growth India",color = "#78c6a3")
ax_P.plot([int(item[-4:]) for item in India.columns[63:].tolist()],India[India.columns[63:]].values[0],linewidth=3,label="Population India",color = "#248277")

ax_G.set_xticks([1961 + i*2 for i in range(0,30)])
ax_G.set_xticklabels([1961 + i*2 for i in range(0,30)])
ax_G.legend(fontsize=18,loc=8)
ax_P.legend(fontsize=18,loc=4)


**The population gap between China and India is narrowing. 👨‍👩‍👧‍👦**  
**This is mainly due to the % growth curve falling much more on the Chinese side than on the Indian side.**  
**It is not long before India overtakes China in total population. 🥇**  
**Another element to highlight this phenomenon is the time when the % growth rate has fallen below the world average.** 

In [ ]:
Growth_AVG = df[df.columns[4:63]].mean()
Pop_AVG = df[df.columns[63:]].mean()

In [ ]:
plt.figure(figsize=[20,20])
ax_G = plt.subplot(211)

fig = plt.figure()
ax_G.set_xlabel("Year",fontsize=10,fontweight=550,labelpad=15)
ax_G.set_ylabel("% Growth",fontsize=10,fontweight=550,labelpad=15)
ax_G.set_title("Comparaison of the % Growth and the population in China with the global average from 1961 to 2019",fontsize=10,fontweight=600)
ax_G.set_xticks([i*2 for i in range(0,30)])
ax_G.set_xticklabels(labels  = [1961 + i*2 for i in range(0,30)])

ax_G.plot([int(item) for item in China.columns[4:63].tolist()],China[China.columns[4:63]].values[0],linewidth=3,label="% Growth China",color = "#ff758f")

ax_G.plot([int(item) for item in China.columns[4:63].tolist()],df[df.columns[4:63]].mean(),linewidth=3,label="% Mean",color = "#48cae4")


ax_G.set_xticks([1961 + i*2 for i in range(0,30)])
ax_G.set_xticklabels([1961 + i*2 for i in range(0,30)])
ax_G.legend(fontsize=15,loc=8)


#Set a figure 
plt.figure(figsize=[20,20])
ax_G2 = plt.subplot(212)

ax_G2.set_xlabel("Year",fontsize=2,fontweight=550,labelpad=15)
ax_G2.set_ylabel("% Growth",fontsize=2,fontweight=550,labelpad=15)
ax_G2.set_title("Comparaison of the % Growth and the population in India with the global average from 1961 to 2019",fontsize=10,fontweight=600)
ax_G2.set_xticks([i*2 for i in range(0,30)])
ax_G2.set_xticklabels(labels  = [1961 + i*2 for i in range(0,30)])

ax_G2.plot([int(item) for item in India.columns[4:63].tolist()],India[India.columns[4:63]].values[0],linewidth=3,label="% Growth India",color = "#78c6a3")

ax_G2.plot([int(item) for item in China.columns[4:63].tolist()],df[df.columns[4:63]].mean(),linewidth=3,label="% Mean",color = "#48cae4")

ax_G2.set_xticks([1961 + i*2 for i in range(0,30)])
ax_G2.set_xticklabels([1961 + i*2 for i in range(0,30)])
ax_G2.legend(fontsize=15,loc=6)


**The graph above shows that China's % growth fell below the world average in 1975. 📆**  
**In contrast, India's growth rate rose above the world average in 1969 and remained above it for more than 35 years (until 2005).**  

**This highlights the significant population growth that India has undergone in recent years, and which risks becoming the most populous country in the world.**  
**An important factor is the one-child policy, or Planned Parenthood policy, the public birth control policy implemented by the People's Republic of China from 1979 to 2015. 👨‍👩‍👧**  

**This reinforces the fall in the % growth over this period.**

# 4) Forecasting world % growth for next years

**Now that we are well advanced on the subject, we can now try to predict what might happen in 20, 30 or even 50 years.**  
**We are going to use different forecasting methods on the global population, taking into account % growth and the world population.**

**Before that, we will convert the data into TimeSeries!** 

In [ ]:
PercentGrowth = pd.melt(World[World.columns[2:61]]).rename(columns={"variable" : "Year","value" : "PercGrowth"})
PercentGrowth["Year"] = PercentGrowth["Year"].map(lambda x : pd.to_datetime(f"{x}-12-31"))
PercentGrowth.head(5)

## Autoregression (AR)  
Autoregressive modeling is training a regression model on the value of response variable itself.  
Autoregressive is made of the word, Auto and Regressive which represents the linear regression on itself (auto).  
In context of time-series forecasting, autoregressive modeling will mean creating the model where the response variable Y will depend upon the previous values of Y at a pre-determined constant time lag. The time lag can be daily (or 2, 3, 4… days), weekly, monthly etc.  
source 🌐 : https://vitalflux.com/autoregressive-ar-models-with-python-examples/

In [ ]:
plt.figure(figsize=[12,8])
plt.title("% Growth of the world population")
plt.xlabel("Year")
plt.ylabel("% Growth")
plt.xticks([i*3 for i in range(0,20)],labels = [1961 + i*3 for i in range(20)])
PercentGrowth['PercGrowth'].plot()

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.ar_model import AutoReg

# Run the test
PG_stationarityTest = adfuller(PercentGrowth['PercGrowth'], autolag='AIC')

# Check the value of p-value
print("P-value: ", PG_stationarityTest[1],", then no stationarity.")

plt.figure(figsize=[12,8])
pacf = plot_pacf(PercentGrowth['PercGrowth'], lags=25)

# Create training and test data
train_data = PercentGrowth['PercGrowth'][:round(len(PercentGrowth['PercGrowth'])*0.8)]
test_data = PercentGrowth['PercGrowth'][round(len(PercentGrowth['PercGrowth'])*0.8):]

In [ ]:
# Instantiate and fit the AR model with training data
ar_model = AutoReg(train_data, lags=6).fit()
print(ar_model.summary())
plt.show()

In [ ]:
# Make the predictions

plt.figure(figsize=[15,10])
pred = ar_model.predict(start=len(train_data), end=(len(PercentGrowth)-1), dynamic=False)

# Plot the prediction vs test data
plt.plot(pred)
plt.plot(test_data, color='red')

In [ ]:
# Make the predictions
plt.figure(figsize=[20,10])
plt.plot(PercentGrowth['PercGrowth'],label="% Growth")
pred = ar_model.predict(start=len(PercentGrowth)-1, end=(len(PercentGrowth)+41), dynamic=False)
plt.plot(pred,color='red',label="Forecast")
plt.xticks([i*4 for i in range(28)],labels = [1961 + i*4 for i in range(28)])
plt.ylabel("Population in Billion",fontsize=15,fontweight=550,labelpad=15)
plt.title("Evolution and Prediction of the % Growth from 1961 to 2061",fontsize=20,fontweight=600)

plt.legend(fontsize=18)

In [ ]:
PopIni = World["Population_2019"].values[0]
Pred_pop_size = []

for percGrowth in pred:
    PopIni = PopIni + PopIni * (percGrowth/100)
    Pred_pop_size.append(PopIni)

In [ ]:
#Set a figure 
fig = plt.figure(figsize=[20,12])
plt.xlabel("Year",fontsize=15,fontweight=550,labelpad=15)
plt.ylabel("Population in 10 billions",fontsize=15,fontweight=550,labelpad=15)
plt.title("Evolution and Prediction of the population size from 1961 to 2061",fontsize=20,fontweight=600)
#ax_P.set_xticks([i*2 for i in range(0,30)])
#ax_P.set_xticklabels(labels  = [1961 + i*2 for i in range(0,30)])

plt.plot([2020 + i for i in range(43)],Pred_pop_size,linewidth=3,label="Prediction",color = "#a4133c")
plt.plot([int(item[-4:]) for item in World.columns[61:].tolist()],World[World.columns[61:]].values[0],linewidth=3,label="Evolution of the population",color = "#0A014F")
plt.vlines(2019, 3000000000, 11000000000, linestyle='--', color='r', label='Start of forecast');
plt.legend(fontsize=18,loc=4)

**A beautiful line is then predicted with more than 10 billion people on Earth by 2050.**

------------------------

## Exponential smoothing

**In order to compare our result with an another method, we are going to use Exponential smoothing for prediction**  

**For our Exponential smoothing, we will use Holt's linear smoothing because our data have a downward trend and no seasonality!**

In [ ]:
PercentGrowth = PercentGrowth.set_index("Year")
PercentGrowth.head(5)

In [ ]:
from statsmodels.tsa.holtwinters import Holt
df = PercentGrowth.copy()
train = df.iloc[:58, :]
test = df.iloc[58:, :]
train.index = pd.to_datetime(train.index,format="%Y")
test.index = pd.to_datetime(test.index,format="%Y")
pred = test.copy()

In [ ]:
model = Holt(np.asarray(train['PercGrowth']))
#model._index = pd.to_datetime(df.index)
fit1 = model.fit(smoothing_level=.7, smoothing_trend=.1)
pred1 = fit1.forecast(len(test)+40)
fit2 = model.fit(smoothing_level=.5, smoothing_trend=.25)
pred2 = fit2.forecast(len(test)+40)
fit3 = model.fit(smoothing_level=.3, smoothing_trend=.15)
pred3 = fit3.forecast(len(test)+40)


fig, ax = plt.subplots(figsize=(20, 12))
ax.plot(df.index, df.values)
for p, f, c in zip((pred1, pred2, pred3),(fit1, fit2, fit3),('#ff7823','#3c763d','c')):
    ax.plot(train.index, f.fittedvalues, color=c)
    ax.plot([pd.to_datetime(f'{year}-12-31T00:00:00.000000000') for year in range(2019,2060)], p, label="alpha="+str(f.params['smoothing_level'])[:4]+", beta="+str(f.params['smoothing_trend'])[:4], color=c)
plt.xlabel("Year",fontsize=15,fontweight=550,labelpad=15)
plt.ylabel("Population in 10 billions",fontsize=15,fontweight=550,labelpad=15)
plt.title("Holt's Exponential Smoothing predictions",fontsize=20,fontweight=600)
plt.legend(fontsize=18)

In [ ]:
PopIni = World["Population_2019"].values[0]
#orange one 
Pred_pop_size1 = []
for percGrowth in pred1:
    PopIni = PopIni + PopIni * (percGrowth/100)
    Pred_pop_size1.append(PopIni)

#green one
PopIni = World["Population_2019"].values[0]
Pred_pop_size2 = []
for percGrowth in pred2:
    PopIni = PopIni + PopIni * (percGrowth/100)
    Pred_pop_size2.append(PopIni)
    
#blue one
PopIni = World["Population_2019"].values[0]
Pred_pop_size3 = []
for percGrowth in pred3: 
    PopIni = PopIni + PopIni * (percGrowth/100)
    Pred_pop_size3.append(PopIni)

In [ ]:
#Set a figure 
fig = plt.figure(figsize=[20,12])
plt.xlabel("Year",fontsize=15,fontweight=550,labelpad=15)
plt.ylabel("Population in 10 billions",fontsize=15,fontweight=550,labelpad=15)
plt.title("Evolution and Prediction of the population size from 1961 to 2061",fontsize=20,fontweight=600)

plt.plot([2020 + i for i in range(41)],Pred_pop_size1,linewidth=3,label="Prediction 1",color = "#ff7823")
plt.plot([2020 + i for i in range(41)],Pred_pop_size2,linewidth=3,label="Prediction 2",color = "#3c763d")
plt.plot([2020 + i for i in range(41)],Pred_pop_size3,linewidth=3,label="Prediction 3 ",color = "c")
plt.plot([int(item[-4:]) for item in World.columns[61:].tolist()],World[World.columns[61:]].values[0],linewidth=3,label="Evolution of the population",color = "#0A014F")
plt.vlines(2019, 3000000000, 11000000000, linestyle='--', color='r', label='Start of forecast');

plt.vlines(2050, 3000000000, 11000000000, linestyle='--', color='b', label='2050');
plt.legend(fontsize=18,loc=4)

**With this method, we find also a result between 10 and 10.5 billions people by 2050 !!**  
**We will then be forced to optimise our land and resources**  
**An other challenge for data scientist ! 📞**

**Thank you for following this notebook! 📓**  
**If you want to give me some advices, don't hesitate! 🙏**  
**We are here to learn ⚡**

## Baptiste 